In [7]:
import os
import glob
import numpy as np
import rasterio
from tqdm.auto import tqdm
from pathlib import Path

In [8]:
base_dir = '/home/nick/Documents/Work code/gimp_segmentation_plugin/sample data'

# this is used to georeference the masks files as it was lossed in GIMP
model_output_folder = 'model output'
image_dir = os.path.join(base_dir,model_output_folder)

# this is the folder with the masks edited in GIMP
colour_mask_folder = 'colour mask'
colour_mask_dir = os.path.join(base_dir,colour_mask_folder)

# this is the output folder
signle_band_masks_folder = 'edited masks' 
signle_band_masks_dir = os.path.join(base_dir,signle_band_masks_folder)
Path(signle_band_masks_dir).mkdir(parents=True, exist_ok=True)

model_output_files = glob.glob(image_dir+'/*tif')
len(model_output_files)

15

In [9]:
# georeference colour masks
for file in model_output_files:
    file_name = os.path.basename(file)
#     open model oputput with rasterio
    src = rasterio.open(file)
#     find corresponding colour mask
    matched_colour_mask = os.path.join(colour_mask_dir,file_name)
#     open colour mask
    src_colour = rasterio.open(matched_colour_mask)
#     grab the colour mask array
    array = src_colour.read()
#     grab a copy of the model oputput metadata
    meta_copy = src.meta
#     change band count 
    meta_copy['count'] = 3
#     export file with georeference
    with rasterio.open(matched_colour_mask, 'w', **meta_copy) as dst:
            dst.write(array)

In [10]:
# open palette file and read colours
rgb_colours = np.loadtxt(os.path.join(base_dir,'Palette.txt')).astype(int)
rgb_colours

array([[230,  25,  75],
       [ 60, 180,  75]])

In [11]:
# get list of colour mask images
colour_mask_files = glob.glob(colour_mask_dir+'/*')
len(colour_mask_files)

15

In [12]:
# convert colour masks into single band masks
for colour_mask in tqdm(colour_mask_files):
    file_name = os.path.basename(colour_mask)
    export_path = os.path.join(signle_band_masks_dir,file_name)
#     open colour mask with rasterio
    colour_mask_src = rasterio.open(colour_mask)
    
    image = colour_mask_src.read()
#     move axis arround so we can use np.all
    image = np.moveaxis(image, 0, -1)
    for colour_numb in range(0,len(rgb_colours)):
        
        image[np.all(image == rgb_colours[colour_numb], axis=-1)] = (colour_numb,colour_numb,colour_numb)
#     move axis back to rasterio expects and grab first band
    image = np.moveaxis(image, 2, 0)[0]
#     expands dims for rasterio
    image = np.expand_dims(image, axis=0)
#     edit metadata
    meta_copy = colour_mask_src.meta
    meta_copy['count'] = 1
    meta_copy['dtype'] = 'uint8'
#     save it out
    with rasterio.open(export_path, 'w', **meta_copy) as dst:
            dst.write(image)

  0%|          | 0/15 [00:00<?, ?it/s]